# Multilayer Tidal Calculations
In this cookbook we will explore how we can use TidalPy's multilayer tidal functions to calculate tidal heating as a function of radius.
Here we are using a propagation matrix technique (SVC16) that is only valid in the incompressible limit.

**References**:
- SVC16 : Sabadini, Vermeerson, & Cambiotti (2016, DOI: [10.1007/978-94-017-7552-6](https://doi.org/10.1007/978-94-017-7552-6).
- HH14  : Henning & Hurford (2014, DOI: [10.1088/0004-637X/789/1/30](https://doi.org/10.1088/0004-637X/789/1/30)).
- TB05  : Tobie et al. (2005), DOI: [10.1016/j.icarus.2005.04.006](https://doi.org/10.1016/j.icarus.2005.04.006).
- ID    : [IcyDwarf Code](https://github.com/MarcNeveu/IcyDwarf/blob/master/IcyDwarf/Thermal.h) written by Marc Neveu

## Build the planet

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interact, widgets
%matplotlib ipympl
np.seterr(divide='raise')

from TidalPy import build_world
from TidalPy.constants import G
from TidalPy.tools.conversions import orbital_motion2semi_a
from TidalPy.utilities.numpy_helper import find_nearest
from TidalPy.rheology.complex_compliance.compliance_models import maxwell_array, sundberg_array
# Load TidalPy's multilayer functions
from TidalPy.tides.multilayer import fundamental_matrix_orderl2, propagate, decompose
from TidalPy.tides.potential.synchronous_low_e import tidal_potential
from TidalPy.tides.multilayer.heating import calc_radial_tidal_heating
from TidalPy.tides.multilayer.stress_strain import calculate_strain

In [2]:
# For this example we will use the Io-Jupiter system
io = build_world('Io')
io.paint()
jupiter = build_world('Jupiter')
tidal_host_mass = jupiter.mass
eccentricity = 0.0041
orbital_freq = 2. * np.pi / (86400. * 1.76914)
orbital_period = 2. * np.pi / orbital_freq
semi_major_axis = orbital_motion2semi_a(orbital_freq, tidal_host_mass, io.mass)

# We won't be using TidalPy's OOP approach (which currently does not use the multi-layer approach,
#    instead relying on a homogenous model broken up by layers), instead we will pull out a few parameters needed
#    for the multilayer calculations.
radius_array = io.radii
surf_area_array = 4. * np.pi * radius_array[1:]**2
volume_array = io.volume_slices
# Depth array skips the first shell since most viscoelastic properties are not defined there.
depth_array = (io.radius - radius_array)[1:]
gravity_array = io.gravities
density_array = io.densities
pressure_array = io.pressures

# We will give the core and mantle a different viscoelastic state, but to do that we need to know what index
#   corresponds to the top of the core.
core_radius_cutoff = find_nearest(radius_array, io.core.radius)
core_radius_cutoff_real = core_radius_cutoff - 1
mantle_N = len(radius_array[core_radius_cutoff:])
core_N = len(radius_array[:core_radius_cutoff])
core_N_real = core_N - 1
total_N = len(radius_array)
total_N_real = total_N - 1

# Determine some global properties
world_radius = radius_array[-1]
world_surf_area = 4. * np.pi * world_radius**2
surface_gravity = gravity_array[-1]
mantle_bulk_density = np.average(density_array[core_radius_cutoff:])

# Setup Io's viscoelastic state - this is not particularly accurate, just meant for demonstration purposes

# All of the viscoelastic properties are not defined at r=0, so we are skipping the inner-most shell. This is done through
#    the "len(radius_array) - 1"

# Tidal dissipation is not too sensitive to bulk modulus so we will keep it constant
bulk_moduli = 200.0e9 * np.ones(len(radius_array) - 1)
bulk_moduli[:core_radius_cutoff_real] = 800.0e9

# Shear modulus and viscosity is much more important, but to keep this example simple we will assume a
#   constant shear and an increasing viscosity towards the surface
shear_moduli = 50.0e9 * np.ones(len(radius_array) - 1)
shear_moduli[:core_radius_cutoff_real] = 80.0e9

viscosities = 1.e50 * np.ones(len(radius_array) - 1)
viscosities[core_radius_cutoff_real:] = np.logspace(23, 14, mantle_N)

# Now we can calculate the complex_compliance of the world. We will just use a Maxwell model here but several
#    others are available.
complex_compliances = sundberg_array(orbital_freq, shear_moduli**(-1), viscosities)
# We will specifically turn off tidal dissipation in the core by setting the imaginary portion of
#    complex compliance = 0
complex_compliances[:core_radius_cutoff_real] = \
    np.real(complex_compliances[:core_radius_cutoff_real]) + 1.e-50j * np.ones(core_N_real)
complex_shears = complex_compliances**(-1)

restrict_propagation_to_mantle = True
if restrict_propagation_to_mantle:
    radius_array = io.radii[core_radius_cutoff:]
    surf_area_array = 4. * np.pi * radius_array**2
    volume_array = io.volume_slices[core_radius_cutoff:]
    # Depth array skips the first shell since most viscoelastic properties are not defined there.
    depth_array = (io.radius - radius_array)
    gravity_array = io.gravities[core_radius_cutoff:]
    density_array = io.densities[core_radius_cutoff:]
    pressure_array = io.pressures[core_radius_cutoff:]
    viscosities = viscosities[core_radius_cutoff_real:]
    complex_compliances = complex_compliances[core_radius_cutoff_real:]
    complex_shears = complex_compliances**(-1)
    shear_moduli = shear_moduli[core_radius_cutoff_real:]
    bulk_moduli = bulk_moduli[core_radius_cutoff_real:]

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [3]:
# Now let's see what this looks like
fig_visco, ax_visco = plt.subplots()
ax_shear = ax_visco.twiny()
ax_comp = ax_visco.twiny()
ax_comp.spines["top"].set_position(("axes", 1.2))

ax_visco.plot(viscosities, depth_array/1000., c='b', ls=':')
ax_shear.plot(shear_moduli, depth_array/1000., c='r', ls='-.')
ax_comp.plot(np.real(complex_shears), depth_array/1000., c='orange', ls='-')
ax_comp.plot(np.imag(complex_shears), depth_array/1000., c='orange', ls='--')
ax_visco.set_ylim(ax_visco.get_ylim()[::-1])
ax_comp.set_ylim(ax_comp.get_ylim()[::-1])
ax_shear.set_ylim(ax_shear.get_ylim()[::-1])

ax_visco.set(ylabel='Depth [km]', xlabel='Viscosity [Pa s]', xscale='log')
ax_shear.set(xlabel='Shear Modulus [Pa]', xscale='linear')
ax_comp.set(xlabel='Complex Shear (Solid=Real, Dash=Imag) [Pa$^{-1}$]', xscale='log')
for ax, color in zip([ax_visco, ax_shear, ax_comp], ['b', 'r', 'orange']):
    if ax is ax_visco:
        ax.spines['bottom'].set_color(color)
    else:
        ax.spines['top'].set_color(color)
    ax.xaxis.label.set_color(color)

fig_visco.tight_layout()
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [4]:
# Find the fundamental matrix; skip the innermost shell as that will be a boundary condition.
Y, Y_inv, derivative_mtx = fundamental_matrix_orderl2(radius_array, complex_shears, density_array, gravity_array)

In [5]:
# Propagate the tidal solution through the world's shells
central_boundary_condition = np.zeros((6, 3), dtype=np.complex128)
# Roberts & Nimmo (2008): Liquid innermost zone.
central_boundary_condition[0, 0] = 0.05 + 0.0j
central_boundary_condition[1, 1] = 0.01 + 0.0j
central_boundary_condition[5, 2] = 1.0 + 0.0j

tidal_y, tidal_y_deriv = propagate(Y, Y_inv, derivative_mtx, central_boundary_condition, world_radius, order_l=2)

In [6]:
# Decompose the tidal solution into useful properties
#    There is a gradient in this function so we lose another shell. Thus the [1:] in viscoelastic properties
radial_sensitivity_to_shear, (k2, h2, l2) = \
    decompose(tidal_y, tidal_y_deriv, radius_array, gravity_array, complex_shears, bulk_moduli, order_l=2)

print('Surface Love & Shida Numbers')
print(f'k_2 = {k2[-1]:.2e}; h_2 = {h2[-1]:.2e}; l_2 = {l2[-1]:.2e}')

# Let's plot these as a function of depth.
fig_love, love_axes = plt.subplots(ncols=2)
fig_love.suptitle('Love and Shida Numbers')

for ax_i, ax in enumerate(love_axes):
    ax_love_h = ax.twiny()
    ax_love_l = ax.twiny()
    ax_love_l.spines["top"].set_position(("axes", 1.2))
    
    if ax_i == 0:
        ax.plot(np.real(k2), depth_array/1000., c='b', ls='-')
        ax_love_h.plot(np.real(h2), depth_array/1000., c='r', ls='-')
        ax_love_l.plot(np.real(l2), depth_array/1000., c='orange', ls='-')
        
        ax.set(ylabel='Depth [km]', xlabel='Re[$k_{2}$]', xscale='linear')
        ax_love_h.set(xlabel='Re[$h_{2}$]', xscale='linear')
        ax_love_l.set(xlabel='Re[$l_{2}$]', xscale='linear')
        ax.set_ylim(ax.get_ylim()[::-1])
        ax_love_h.set_ylim(ax_love_h.get_ylim()[::-1])
        ax_love_l.set_ylim(ax_love_l.get_ylim()[::-1])
    else:
        ax.plot(-np.imag(k2), depth_array/1000., c='b', ls='-')
        ax_love_h.plot(-np.imag(h2), depth_array/1000., c='r', ls='-')
        ax_love_l.plot(-np.imag(l2), depth_array/1000., c='orange', ls='-')

        ax.set(ylabel='Depth [km]', xlabel='-Im[$k_{2}$]', xscale='log')
        ax_love_h.set(xlabel='-Im[$h_{2}$]', xscale='log')
        ax_love_l.set(xlabel='-Im[$l_{2}$]', xscale='log')
        ax.set_ylim(ax.get_ylim()[::-1])
        ax_love_h.set_ylim(ax_love_h.get_ylim()[::-1])
        ax_love_l.set_ylim(ax_love_l.get_ylim()[::-1])

    for ax2, color in zip([ax, ax_love_h, ax_love_l], ['b', 'r', 'orange']):
        if ax2 is ax_visco:
            ax2.spines['bottom'].set_color(color)
        else:
            ax2.spines['top'].set_color(color)
        ax2.xaxis.label.set_color(color)

fig_love.tight_layout()
plt.show()

Surface Love & Shida Numbers
k_2 = 2.95e-01-4.02e-01j; h_2 = 5.76e-01-7.42e-01j; l_2 = 1.32e-01-1.70e-01j


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## Calculate Tidal Displacement
Utilizing the simplified tidal potential (low eccentricity, no obliquity, spin-synchronous).

In [7]:
# Define latitude and longitude domain
colatitude = np.linspace(0.1, 179.9, 25)
latitude = colatitude - 90.
longitude = np.linspace(0., 360., 30)
longitude_r = np.radians(longitude)
colatitude_r = np.radians(colatitude)
time_domain = np.linspace(0., orbital_period, 50)

rad_mtx, time_mtx, long_mtx, lat_mtx = \
    np.meshgrid(radius_array, time_domain, longitude_r, colatitude_r, indexing='ij')

In [8]:
# Calculate the simplified tidal potential
potential, potential_dtheta, potential_dphi, potential_d2theta, potential_d2phi, potential_dtheta_dphi = \
    tidal_potential(rad_mtx, long_mtx, lat_mtx, orbital_freq, eccentricity, time_mtx)

# Scale by the tidal solutions
u_r = radial_displacement = tidal_y[0, -1] * potential
u_th = polar_displacement = tidal_y[2, -1] * potential_dtheta
u_phi = azimuthal_displacement = tidal_y[2, -1] * potential_dphi / np.sin(lat_mtx)

In [9]:
# Plot results
fig_disp, axes_disp = plt.subplots(ncols=3, figsize=(8, 5))
plt.subplots_adjust(wspace=.5)
ax_disp_r = axes_disp[0]
ax_disp_th = axes_disp[1]
ax_disp_phi = axes_disp[2]

def update_fig(time_i=0, radius_idx=0):
    orbital_period_now_displacement = time_domain[time_i] / orbital_period
    depth_now_displacement = radius_array[1:][radius_idx]
    ax_disp_r.clear()
    ax_disp_th.clear()
    ax_disp_phi.clear()
    fig_disp.suptitle(f'{orbital_period_now_displacement:0.2f} Orbital Period; {depth_now_displacement/1000:0.0f} km')
    
    ax_disp_r.set(ylabel='Latitude [$\deg$]', xlabel='Longitude [$\deg$]',
                  title='Radial Displacement', xlim=(0, 360), ylim=(-90, 90))
    ax_disp_th.set(ylabel='Latitude [$\deg$]', xlabel='Longitude [$\deg$]',
                   title='Polar Displacement', xlim=(0, 360), ylim=(-90, 90))
    ax_disp_phi.set(ylabel='Latitude [$\deg$]', xlabel='Longitude [$\deg$]',
                    title='Azimuthal Displacement', xlim=(0, 360), ylim=(-90, 90))
    
    ax_disp_r.contourf(longitude, latitude, u_r[radius_idx, time_i, :, :].T, 15)
    ax_disp_th.contourf(longitude, latitude, u_th[radius_idx, time_i, :, :].T, 15)
    ax_disp_phi.contourf(longitude, latitude, u_phi[radius_idx, time_i, :, :].T, 15)
    fig_disp.tight_layout()

interact(update_fig, time_i=(0, len(time_domain)-1, 1),
         radius_idx=widgets.IntSlider(min=0, max=len(radius_array[1:])-1, step=1, value=len(radius_array[1:])-1))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

interactive(children=(IntSlider(value=0, description='time_i', max=49), IntSlider(value=48, description='radiu…

<function __main__.update_fig(time_i=0, radius_idx=0)>

## Orbit-Averaged, Tidal Heating as Function of Depth
Here we will use a modified version of the heating(radius) equations found in TB05. 

In [10]:
# We will use the radial sensitivity to shear calculated in a previous step.
vol_heating_as_func_radius = \
    calc_radial_tidal_heating(eccentricity, orbital_freq, semi_major_axis,
                              tidal_host_mass,
                              radius_array, radial_sensitivity_to_shear,
                              complex_shears, order_l=2)

print(f'Total Heating {sum(vol_heating_as_func_radius * volume_array ):0.2e} W')
# Plot result
fig_heat_depth_mu, ax_heat_depth_mu = plt.subplots()
ax_heat_depth_mu.set(ylabel='Depth [km]', xlabel='Volumetric Heating Rate [W m$^{-3}$]', xscale='log')
ax_heat_depth_mu.plot(vol_heating_as_func_radius, depth_array / 1000.)
ax_heat_depth_mu.set_ylim(ax_heat_depth_mu.get_ylim()[::-1])
fig_heat_depth_mu.tight_layout()
plt.show()

Total Heating 1.88e+15 W


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## Strain and Stress Tensors
Here we will calculate the strain and stress tensors using the methods described in TB05 and B13

In [11]:
# We need to recast the tidal y solution into the correct dimensions
#    (it does not care about long/lat or time - at least in this context).
tidal_y_higher_dim = np.zeros((6, *rad_mtx.shape), dtype=np.complex128)
tidal_y_deriv_higher_dim = np.zeros((6, *rad_mtx.shape), dtype=np.complex128)

complex_shears_higher_dim, _, _, _ = \
        np.meshgrid(complex_shears, time_domain, longitude_r, colatitude_r, indexing='ij')
bulk_moduli_higher_dim, _, _, _ = \
        np.meshgrid(bulk_moduli, time_domain, longitude_r, colatitude_r, indexing='ij')
radius_array_higher_dim, _, _, _ = \
        np.meshgrid(radius_array, time_domain, longitude_r, colatitude_r, indexing='ij')
volume_array_higher_dim, _, _, _ = \
        np.meshgrid(volume_array, time_domain, longitude_r, colatitude_r, indexing='ij')
for i in range(6):
    tidal_y_higher_dim[i, :, :, :, :], _, _, _ = \
        np.meshgrid(tidal_y[i, :], time_domain, longitude_r, colatitude_r, indexing='ij')
    tidal_y_deriv_higher_dim[i, :, :, :, :], _, _, _ = \
        np.meshgrid(tidal_y_deriv[i, :], time_domain, longitude_r, colatitude_r, indexing='ij')

# Calculate the simplified tidal potential
potential_STR, potential_dtheta_STR, potential_dphi_STR, potential_d2theta_STR, potential_d2phi_STR, potential_dtheta_dphi_STR = \
    tidal_potential(rad_mtx, long_mtx, lat_mtx, orbital_freq, eccentricity, time_mtx)

# Use the tidal solutions to scale the potential and find the strains
strain_rr, strain_thth, strain_phph, strain_rth, strain_rph, strain_thph = \
    calculate_strain(potential_STR, potential_dtheta_STR, potential_dphi_STR,
                     potential_d2theta_STR, potential_d2phi_STR,
                     potential_dtheta_dphi_STR, tidal_y_higher_dim, tidal_y_deriv_higher_dim,
                     colatitude=lat_mtx, radius=radius_array_higher_dim, shear_moduli=complex_shears_higher_dim)

# Use strains to calculate stress
trace = strain_rr + strain_thth + strain_phph
kk_term = (bulk_moduli_higher_dim - (2. / 3.) * complex_shears_higher_dim) * trace

# Find Stresses
stress_rr = (2. * complex_shears_higher_dim * strain_rr) + kk_term
stress_thth = (2. * complex_shears_higher_dim * strain_thth) + kk_term
stress_phph = (2. * complex_shears_higher_dim * strain_phph) + kk_term
stress_rth = 2. * complex_shears_higher_dim * strain_rth 
stress_rph = 2. * complex_shears_higher_dim * strain_rph
stress_thph = 2. * complex_shears_higher_dim * strain_thph

### Stress and Strain over time

In [12]:
# Plot results
fig_strain_time, axes_stress_strain_time = plt.subplots(ncols=3, nrows=4, figsize=(8, 6))
ax_strain_r = axes_stress_strain_time[0, 0]
ax_strain_th = axes_stress_strain_time[0, 1]
ax_strain_phi = axes_stress_strain_time[0, 2]
ax_strain_rth = axes_stress_strain_time[1, 0]
ax_strain_rphi = axes_stress_strain_time[1, 1]
ax_strain_thphi = axes_stress_strain_time[1, 2]
ax_stress_r = axes_stress_strain_time[2, 0]
ax_stress_th = axes_stress_strain_time[2, 1]
ax_stress_phi = axes_stress_strain_time[2, 2]
ax_stress_rth = axes_stress_strain_time[3, 0]
ax_stress_rphi = axes_stress_strain_time[3, 1]
ax_stress_thphi = axes_stress_strain_time[3, 2]

def update_fig(time_i=0, radius_idx=0, show_imaginary=False):
    orbital_period_now_displacement = time_domain[time_i] / orbital_period
    depth_now_displacement = radius_array[1:][radius_idx]
    
    str_pre = ''
    str_post = ''
    def scale_func(arry):
        return arry
    if show_imaginary:
        str_pre = 'Im['
        str_post = ']'
        def scale_func(arry):
            return np.imag(arry)
    
    e_rr = strain_rr[radius_idx, time_i, :, :]
    e_thth = strain_thth[radius_idx, time_i, :, :]
    e_phph = strain_phph[radius_idx, time_i, :, :]
    e_rth = 2. * strain_rth[radius_idx, time_i, :, :]
    e_rph = 2. * strain_rph[radius_idx, time_i, :, :]
    e_thph = 2. * strain_thph[radius_idx, time_i, :, :]
    o_rr = stress_rr[radius_idx, time_i, :, :]
    o_thth = stress_thth[radius_idx, time_i, :, :]
    o_phph = stress_phph[radius_idx, time_i, :, :]
    o_rth = 2. * stress_rth[radius_idx, time_i, :, :]
    o_rph = 2. * stress_rph[radius_idx, time_i, :, :]
    o_thph = 2. * stress_thph[radius_idx, time_i, :, :]
    
    ax_strain_r.clear()
    ax_strain_th.clear()
    ax_strain_phi.clear()
    ax_strain_rth.clear()
    ax_strain_rphi.clear()
    ax_strain_thphi.clear()
    ax_stress_r.clear()
    ax_stress_th.clear()
    ax_stress_phi.clear()
    ax_stress_rth.clear()
    ax_stress_rphi.clear()
    ax_stress_thphi.clear()
    
    fig_strain_time.suptitle(f'{orbital_period_now_displacement:0.2f} Orbital Period; {depth_now_displacement/1000:0.0f} km')
    
    ax_strain_r.set(ylabel='Latitude [$\deg$]',
                    title=str_pre+'$\\epsilon_{rr}$'+str_post, xlim=(0, 360), ylim=(-90, 90))
    ax_strain_th.set(title=str_pre+'$\\epsilon_{\\theta\\theta}$'+str_post, xlim=(0, 360), ylim=(-90, 90))
    ax_strain_phi.set(title=str_pre+'$\\epsilon_{\\phi\\phi}$'+str_post, xlim=(0, 360), ylim=(-90, 90))
    ax_strain_rth.set(ylabel='Latitude [$\deg$]',
                      title=str_pre+'$\\epsilon_{r\\theta}$'+str_post, xlim=(0, 360), ylim=(-90, 90))
    ax_strain_rphi.set(title=str_pre+'$\\epsilon_{r\\phi}$'+str_post, xlim=(0, 360), ylim=(-90, 90))
    ax_strain_thphi.set(title=str_pre+'$\\epsilon_{\\theta\\phi}$'+str_post, xlim=(0, 360), ylim=(-90, 90))
    
    ax_stress_r.set(ylabel='Latitude [$\deg$]',
                      title=str_pre+'$\\sigma_{rr}$'+str_post, xlim=(0, 360), ylim=(-90, 90))
    ax_stress_th.set(title=str_pre+'$\\sigma_{\\theta\\theta}$'+str_post, xlim=(0, 360), ylim=(-90, 90))
    ax_stress_phi.set(title=str_pre+'$\\sigma_{\\phi\\phi}$'+str_post, xlim=(0, 360), ylim=(-90, 90))
    ax_stress_rth.set(ylabel='Latitude [$\deg$]', xlabel='Longitude [$\deg$]',
                      title=str_pre+'$\\sigma_{r\\theta}$'+str_post, xlim=(0, 360), ylim=(-90, 90))
    ax_stress_rphi.set(xlabel='Longitude [$\deg$]',
                       title=str_pre+'$\\sigma_{r\\phi}$'+str_post, xlim=(0, 360), ylim=(-90, 90))
    ax_stress_thphi.set(xlabel='Longitude [$\deg$]',
                        title=str_pre+'$\\sigma_{\\theta\\phi}$'+str_post, xlim=(0, 360), ylim=(-90, 90))
    
    ax_strain_r.contourf(longitude, latitude, scale_func(e_rr.T), 10)
    ax_strain_th.contourf(longitude, latitude, scale_func(e_thth.T), 10)
    ax_strain_phi.contourf(longitude, latitude, scale_func(e_phph.T), 10)
    ax_strain_rth.contourf(longitude, latitude, scale_func(e_rth.T), 10)
    ax_strain_rphi.contourf(longitude, latitude, scale_func(e_rph.T), 10)
    ax_strain_thphi.contourf(longitude, latitude, scale_func(e_thph.T), 10)
    ax_stress_r.contourf(longitude, latitude, scale_func(o_rr.T), 10)
    ax_stress_th.contourf(longitude, latitude, scale_func(o_thth.T), 10)
    ax_stress_phi.contourf(longitude, latitude, scale_func(o_phph.T), 10)
    ax_stress_rth.contourf(longitude, latitude, scale_func(o_rth.T), 10)
    ax_stress_rphi.contourf(longitude, latitude, scale_func(o_rph.T), 10)
    ax_stress_thphi.contourf(longitude, latitude, scale_func(o_thph.T), 10)
    
    fig_strain_time.tight_layout()
    plt.subplots_adjust(wspace=.2, hspace=0.55)

interact(update_fig, time_i=(0, len(time_domain)-1, 1),
         radius_idx=widgets.IntSlider(min=0, max=len(radius_array[1:])-1, step=1, value=len(radius_array[1:])-1),
         show_imaginary=False)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

interactive(children=(IntSlider(value=0, description='time_i', max=49), IntSlider(value=48, description='radiu…

<function __main__.update_fig(time_i=0, radius_idx=0, show_imaginary=False)>

### Stress over depth
**Latitude: 0, Longitude: 0**

In [13]:
latitude_idx = find_nearest(latitude, 25.)
longitude_idx = find_nearest(longitude, 120.)
time_index = 10

show_real = False
show_imag = False
show_abs = False

def scale_func(arry):
    return arry

str_pre = ''
str_post = ''
if show_real:
    str_pre = 'Re['
    str_post = ']'
    def scale_func(arry):
        return np.real(arry)
elif show_imag:
    str_pre = 'Im['
    str_post = ']'
    def scale_func(arry):
        return np.imag(arry)
elif show_abs:
    str_pre = '|'
    str_post = '|'
    def scale_func(arry):
        return np.abs(arry)
    

# Plot results
fig_strain_depth, axes_strain_depth = plt.subplots(ncols=3, nrows=2, figsize=(9, 5))
plt.subplots_adjust(wspace=.6, hspace=0.5)
ax_strain_depth_r = axes_strain_depth[0, 0]
ax_strain_depth_th = axes_strain_depth[0, 1]
ax_strain_depth_phi = axes_strain_depth[0, 2]
ax_strain_depth_rth = axes_strain_depth[1, 0]
ax_strain_depth_rphi = axes_strain_depth[1, 1]
ax_strain_depth_thphi = axes_strain_depth[1, 2]
ax_stress_depth_r = ax_strain_depth_r.twiny()
ax_stress_depth_th = ax_strain_depth_th.twiny()
ax_stress_depth_phi = ax_strain_depth_phi.twiny()
ax_stress_depth_rth = ax_strain_depth_rth.twiny()
ax_stress_depth_rphi = ax_strain_depth_rphi.twiny()
ax_stress_depth_thphi = ax_strain_depth_thphi.twiny()

ax_strain_depth_r.set(ylabel='Depth [km]', xlabel=str_pre+'$\\epsilon_{rr}$'+str_post)
ax_strain_depth_th.set(xlabel=str_pre+'$\\epsilon_{\\theta\\theta}$'+str_post)
ax_strain_depth_phi.set(xlabel=str_pre+'$\\epsilon_{\\phi\\phi}$'+str_post)
ax_strain_depth_rth.set(ylabel='Depth [km]', xlabel=str_pre+'$\\epsilon_{r\\theta}$'+str_post)
ax_strain_depth_rphi.set(xlabel=str_pre+'$\\epsilon_{r\\phi}$'+str_post)
ax_strain_depth_thphi.set(xlabel=str_pre+'$\\epsilon_{\\theta\\phi}$'+str_post)
ax_stress_depth_r.set(xlabel=str_pre+'$\\sigma_{rr}$'+str_post)
ax_stress_depth_th.set(xlabel=str_pre+'$\\sigma_{\\theta\\theta}$'+str_post)
ax_stress_depth_phi.set(xlabel=str_pre+'$\\sigma_{\\phi\\phi}$'+str_post)
ax_stress_depth_rth.set(xlabel=str_pre+'$\\sigma_{r\\theta}$'+str_post)
ax_stress_depth_rphi.set(xlabel=str_pre+'$\\sigma_{r\\phi}$'+str_post)
ax_stress_depth_thphi.set(xlabel=str_pre+'$\\sigma_{\\theta\\phi}$'+str_post)

e_rr_depth = strain_rr[:, time_index, longitude_idx, latitude_idx]
e_thth_depth = strain_thth[:, time_index, longitude_idx, latitude_idx]
e_phph_depth = strain_phph[:, time_index, longitude_idx, latitude_idx]
e_rth_depth = 2. * strain_rth[:, time_index, longitude_idx, latitude_idx]
e_rph_depth = 2. * strain_rph[:, time_index, longitude_idx, latitude_idx]
e_thph_depth = 2. * strain_thph[:, time_index, longitude_idx, latitude_idx]
o_rr_depth = stress_rr[:, time_index, longitude_idx, latitude_idx]
o_thth_depth = stress_thth[:, time_index, longitude_idx, latitude_idx]
o_phph_depth = stress_phph[:, time_index, longitude_idx, latitude_idx]
o_rth_depth = 2. * stress_rth[:, time_index, longitude_idx, latitude_idx]
o_rph_depth = 2. * stress_rph[:, time_index, longitude_idx, latitude_idx]
o_thph_depth = 2. * stress_thph[:, time_index, longitude_idx, latitude_idx]

ax_strain_depth_r.plot(scale_func(e_rr_depth), depth_array / 1000., 'b-')
ax_strain_depth_th.plot(scale_func(e_thth_depth), depth_array / 1000., 'b-')
ax_strain_depth_phi.plot(scale_func(e_phph_depth), depth_array / 1000., 'b-')
ax_strain_depth_rth.plot(scale_func(e_rth_depth), depth_array / 1000., 'b-')
ax_strain_depth_rphi.plot(scale_func(e_rph_depth), depth_array / 1000., 'b-')
ax_strain_depth_thphi.plot(scale_func(e_thph_depth), depth_array / 1000., 'b-')
ax_stress_depth_r.plot(scale_func(o_rr_depth), depth_array / 1000., 'r-')
ax_stress_depth_th.plot(scale_func(o_thth_depth), depth_array / 1000., 'r-')
ax_stress_depth_phi.plot(scale_func(o_phph_depth), depth_array / 1000., 'r-')
ax_stress_depth_rth.plot(scale_func(o_rth_depth), depth_array / 1000., 'r-')
ax_stress_depth_rphi.plot(scale_func(o_rph_depth), depth_array / 1000., 'r-')
ax_stress_depth_thphi.plot(scale_func(o_thph_depth), depth_array / 1000., 'r-')

for ax in [ax_strain_depth_r, ax_strain_depth_th, ax_strain_depth_phi,
           ax_strain_depth_rth, ax_strain_depth_rphi, ax_strain_depth_thphi]:
    
    ax.set_ylim(ax.get_ylim()[::-1])
    ax.spines['bottom'].set_color('blue')
    ax.xaxis.label.set_color('blue')
    ax.tick_params(axis='x', colors='blue')

for ax in [ax_stress_depth_r, ax_stress_depth_th, ax_stress_depth_phi,
           ax_stress_depth_rth, ax_stress_depth_rphi, ax_stress_depth_thphi]:
    
#     ax.set_ylim(ax.get_ylim()[::-1])
    ax.spines['top'].set_color('red')
    ax.xaxis.label.set_color('red')
    ax.tick_params(axis='x', colors='red')
    
fig_strain_depth.tight_layout()
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## Tidal Heating Rate

In [14]:
# Calculate Tidal Heating, using two methods
strain_trace = strain_rr + strain_thth + strain_phph

averaged_strain_sqrd = \
    strain_rr * np.conj(strain_rr) + \
    strain_thth * np.conj(strain_thth) + \
    strain_phph * np.conj(strain_phph) + \
    2. * strain_rth * np.conj(strain_rth) + \
    2. * strain_rph * np.conj(strain_rph) + \
    2. * strain_thph * np.conj(strain_thph)

heating_method_2 = (orbital_freq / 2.) * (
    np.imag(stress_rr) * np.real(strain_rr) - np.real(stress_rr) * np.imag(strain_rr) +
    np.imag(stress_thth) * np.real(strain_thth) - np.real(stress_thth) * np.imag(strain_thth) +
    np.imag(stress_phph) * np.real(strain_phph) - np.real(stress_phph) * np.imag(strain_phph) + 
    2. * (np.imag(stress_rth) * np.real(strain_rth) - np.real(stress_rth) * np.imag(strain_rth)) + 
    2. * (np.imag(stress_rph) * np.real(strain_rph) - np.real(stress_rph) * np.imag(strain_rph)) + 
    2. * (np.imag(stress_thph) * np.real(strain_thph) - np.real(stress_thph) * np.imag(strain_thph)))

heating_method_1 = orbital_freq * np.imag(complex_shears_higher_dim) * \
    (averaged_strain_sqrd - (1. / 3.) * np.abs(trace)**2) * volume_array_higher_dim
heating_method_2 *= volume_array_higher_dim

In [15]:
# Setup Figure
fig_heat, axes_heat = plt.subplots(ncols=2, figsize=(9, 4))
plt.subplots_adjust(wspace=4, hspace=0.3)

axis_heat = axes_heat[0]
axis_heat_depth = axes_heat[1]

cb_heat = None

def update_fig(time_i=0, r_index =-1,
               sum_depth = False, fix_negatives = False, show_flux = True, heat_method_1 = True,
               fixed_long=0., fixed_lat=0., orbit_average=False):
    
    global cb_heat
    
    # Determine heating method to use
    if heat_method_1: 
        heating = np.copy(heating_method_1)
        heating_at_depth = np.copy(heating_method_1)
    else:
        heating = np.copy(heating_method_2)
        heating_at_depth = np.copy(heating_method_2)
        
    if fix_negatives:
        # Check for negatives
        heating[heating < 0.] = 0.
        heating_at_depth[heating_at_depth < 0.] = 0.
        
    # Determine if we are summing over the depth of the world
    if sum_depth:
        heating = np.sum(heating, axis=0)
    else:
        heating = heating[r_index]
    
    # Determine if we are averageing over the orbit
    # FIXME: This is probably not the right way to average?
    if orbit_average:
        heating = np.average(heating, axis=0)
        heating_at_depth = np.average(heating_at_depth, axis=1)
    else:
        heating = heating[time_i]
        heating_at_depth = heating_at_depth[:, time_i]
    
    # Plot Results
    axis_heat.clear()
    axis_heat_depth.clear()
    plt.subplots_adjust(wspace=.1, hspace=0.3)
    
    # Set figure title
    orbital_period_now_displacement = time_domain[time_i] / orbital_period
    depth_now_displacement = radius_array[1:][r_index]
    fig_heat.suptitle(f'{orbital_period_now_displacement:0.2f} Orbital Period; {depth_now_displacement/1000:0.0f} km')
    
    if cb_heat is not None:
        cb_heat.remove()
    axis_heat.set(title='Tidal Heating [W]')
    if show_flux:
        if sum_depth:
            heating_to_plot = heating / world_surf_area
        else:
            heating_to_plot = heating / (4. * np.pi * radius_array[1:][r_index]**2)
        axis_heat.set(title='Surface Heating Flux [W m-2]')
    else:
        heating_to_plot = heating
    axis_heat.set(ylabel='Latitude [$\deg$]', xlabel='Longitude [$\deg$]', xlim=(0, 360), ylim=(-90, 90))
    
    cb_data = axis_heat.contourf(longitude, latitude, heating_to_plot.T, 15)
    cb_heat = plt.colorbar(cb_data, ax=axis_heat)
    
    # plot depth at sub-stellar point
    lat_indx = find_nearest(latitude, fixed_lat)
    long_indx = find_nearest(longitude, fixed_long)
    axis_heat_depth.plot(heating_at_depth[:, long_indx, lat_indx] / volume_array, depth_array/1000.)
    axis_heat_depth.set(xlabel='Tidal Heating [W m-3]', ylabel='Depth [km]', xscale='log')
    axis_heat_depth.set_ylim(axis_heat_depth.get_ylim()[::-1])
    
    axis_heat.plot([longitude[long_indx]], [latitude[lat_indx]], marker='x', c='r')
    
    fig_heat.tight_layout()

interact(update_fig, time_i=(0, len(time_domain)-1, 1),
         r_index=widgets.IntSlider(min=0, max=len(radius_array[1:])-1, step=1, value=len(radius_array[1:])-1),
         sum_depth=False, show_flux=True, heat_method_1=True, orbit_average=False,
         fixed_long=widgets.IntSlider(min=0, max=360, step=30, value=0),
         fixed_lat=widgets.IntSlider(min=-90, max=90, step=30, value=0))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

interactive(children=(IntSlider(value=0, description='time_i', max=49), IntSlider(value=48, description='r_ind…

<function __main__.update_fig(time_i=0, r_index=-1, sum_depth=False, fix_negatives=False, show_flux=True, heat_method_1=True, fixed_long=0.0, fixed_lat=0.0, orbit_average=False)>

## Performance
Here we are just doing a simple check on how the propagation functions perform

In [16]:
run_performance = False

if run_performance:
    print('Building Matricies')
    %timeit fundamental_matrix_orderl2(radius_array[1:], complex_shears[1:], density_array[1:], gravity_array[1:])

    print('Propagating Solutions')
    %timeit propagate(Y, Y_inv, central_boundary_condition, world_radius, order_l=2)

    print('Decomposition')
    %timeit decompose(tidal_y, radius_array[1:], gravity_array[1:], complex_shears[2:], bulk_moduli[2:], order_l=2)